In [70]:
import pickle
import os
import pandas as pd
from glob import glob
from tqdm import tqdm

In [71]:
# # 删除问题行
# metadata_path = "/data_nas/GoogleSV/metadata/China/HongKong/pano_2024-08-17 13:11:09.125553_23512.p"
# metadata = pd.read_pickle(metadata_path)
# error_line = metadata[metadata['panoid'] == "AF1QipOEQeUmG17kqli1N84XJmoOdliKcXrxp64sheE6"]
# metadata = metadata.drop(error_line.index).reset_index(drop=True)
# metadata.to_pickle(metadata_path)

In [72]:
dataset_path = "/data1/cehou_data/LLM_safety/img_text_data/baseline"
dataset = os.listdir(dataset_path)

In [73]:
glob(f'/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_*')

['/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241028_113308_7601_7650.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241028_085420_7401_7600.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241028_131438_7651_7700.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241028_135927_7651_7800.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241028_115740_7601_7700.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241028_124418_7601_7800.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241028_122057_7601_7750.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_basel

In [74]:
glob(f'{dataset_path}/dataset_baseline_*')

['/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241026_153732_5951_7400.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_3851.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241024_115324_5000.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_PlacePulse_20241024_205514_5950.pkl']

In [75]:
# for i,file in enumerate(glob(f'{dataset_path}/dataset_baseline_*')):
for i,file in enumerate(glob(f'/data2/cehou/LLM_safety/img_text_data/baseline/dataset_baseline_*')):
    with open(file, "rb") as f:
        data = pickle.load(f)
        print(len(data))
        if i == 0:
            all_data = data
        else:
            all_data.extend(data)
print(f"total data number: {len(all_data)}")

950
1450
1149
total data number: 3549


In [76]:
# 使用字典来删除重复项，同时保持顺序
unique_all_data = list({item['GSV_idx']: item for item in all_data}.values())
# all_data = unique_all_data
print(f"Number of unique items in all_data: {len(unique_all_data)}")
all_data_df = pd.DataFrame(unique_all_data)

Number of unique items in all_data: 3549


In [77]:
safety_perception = pd.read_csv("/data_nas/cehou/LLM_safety/PlacePulse2.0/image_perception.csv")
safety_perception.rename(columns={'Image_ID': 'GSV_name'}, inplace=True)
# safety_perception = safety_perception.drop_duplicates(subset=['GSV_name']).reset_index(drop=True)

def retidy_oneline(GSV_id):
    aaa = safety_perception[safety_perception['GSV_name'] == GSV_id].T
    aaa.columns = aaa.iloc[1]
    aaa = aaa.iloc[2:,:]
    aaa['GSV_name'] = GSV_id
    return aaa

In [78]:
for i, line in tqdm(all_data_df.iterrows()):
    tem = retidy_oneline(line['GSV_name'])
    # safety_perception = safety_perception.merge(tem, how='left', on='GSV_name')
    if i == 0:
        all_df = tem
    else:
        all_df = pd.concat([all_df, tem], axis=0)


3549it [01:20, 44.16it/s]


In [79]:
all_df

Category,safety,lively,beautiful,wealthy,depressing,boring,GSV_name
Q_Value,0.558899,0.416667,0.555556,0.444444,0.333333,0.222222,50f44560fdc9f065f0003461
Q_Value,0.333333,0.430556,0.5,0.611111,0.666667,0.486111,50f44560fdc9f065f0003462
Q_Value,0.535714,0.354026,0.305556,0.305556,0.5,0.609788,50f44560fdc9f065f0003464
Q_Value,0.409921,0.399647,0.438889,0.573333,0.507407,0.666667,50f44560fdc9f065f0003465
Q_Value,0.494838,0.30754,0.416667,0.455556,0.333333,0.601852,50f44560fdc9f065f0003466
...,...,...,...,...,...,...,...
Q_Value,0.446131,0.450198,0.575661,0.333333,0.573148,0.386111,50f44560fdc9f065f0003453
Q_Value,0.3,0.663889,0.434127,0.666667,0.530101,0.333333,50f44560fdc9f065f0003454
Q_Value,0.31121,0.333333,0.666667,0.416667,0.366667,0.5,50f44560fdc9f065f0003456
Q_Value,0.510216,0.366523,0.533333,0.522222,0.412037,0.4,50f44560fdc9f065f0003457


In [80]:
all_data_df = all_data_df.merge(all_df, how='left', on='GSV_name')

In [82]:
def get_qa(text_description):
    answer = [i.split("<\\s> [INST]")[0] for i in text_description.split(" [/INST] ")][1:]
    question = [text_description.split(" [/INST] ")[0][7:]] + [i.split("<\\s> [INST]")[1] for i in text_description.split(" [/INST] ")[1:-1]]
    return answer, question

def text_processing(dataset, data_type):
    '''
    data_type: 'baseline' or 'others'
    example: text_processing("/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_1401.pkl", 'baseline')
    '''
    # dataset = pd.read_pickle(dataset)
    text_description = [dataset[i]['text_description'] for i in range(len(dataset))]

    answer_list = [get_qa(text_description[i])[0] for i in range(len(text_description))]
    question_list = [get_qa(text_description[i])[1] for i in range(len(text_description))]
    if data_type == 'baseline':
        text_description_new = [answer_list[i][1] for i in range(len(text_description))]
        for i in range(len(dataset)):
            dataset[i]['text_description_short'] = text_description_new[i]
    else:
        print('not implemented')

    # with open(dataset_path, 'wb') as f:
    #     pickle.dump(dataset, f)
    return None


In [83]:
data_config = dataset[0].split("_")
save_filename = f"/data2/cehou/LLM_safety/img_text_data/baseline/dataset_{data_config[1]}_{data_config[2]}_{data_config[3]}_{data_config[4]}_{len(all_data_df)}_2.pkl"
all_data_df.to_pickle(save_filename)
print(f"save the data into {save_filename}")

save the data into /data2/cehou/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_3549_2.pkl


In [87]:
tidyed_data_list = glob(f'/data2/cehou/LLM_safety/img_text_data/baseline/tidyed/*')
for i, data in enumerate(tidyed_data_list):
    print(f"processing {data}")
    if i == 0:
        data_final = pd.read_pickle(data)
    else:
        data_final = pd.concat([data_final, pd.read_pickle(data)], axis=0)
    

processing /data2/cehou/LLM_safety/img_text_data/baseline/tidyed/dataset_baseline_baseline_baseline_baseline_3851.pkl
processing /data2/cehou/LLM_safety/img_text_data/baseline/tidyed/dataset_baseline_baseline_baseline_baseline_3549_2.pkl
processing /data2/cehou/LLM_safety/img_text_data/baseline/tidyed/dataset_baseline_baseline_baseline_baseline_400_2.pkl
processing /data2/cehou/LLM_safety/img_text_data/baseline/tidyed/dataset_baseline_baseline_baseline_baseline_450_2.pkl


In [90]:
data_final = data_final.drop_duplicates().reset_index(drop=True)
# data_final.rename(columns={'GSV_name': 'Image_ID'}, inplace=True)

In [129]:
data_final.iloc[:,:-6]

,GSV_idx,Image_ID,GSV_path,text_description,age,gender,location,event,text_description_short
0,501,50f42b7bfdc9f065f00013ba,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a reside...
1,502,50f42b7bfdc9f065f00013bb,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a street...
2,503,50f42b7bfdc9f065f00013bd,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,"In the panoramic street view image, there are ..."
3,504,50f42b7bfdc9f065f00013be,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The street scene in the panoramic street view ...
4,505,50f42b7bfdc9f065f00013c0,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a quiet ...
...,...,...,...,...,...,...,...,...,...
7845,7846,50f55e33fdc9f065f0004b09,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a street...
7846,7847,50f55e33fdc9f065f0004b0b,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a tree-l...
7847,7848,50f55e33fdc9f065f0004b0d,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a street...
7848,7849,50f55e33fdc9f065f0004b0e,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a street...


In [117]:
true_score_value = pd.read_csv('/data2/cehou/LLM_safety/PlacePulse2.0/metadata/pulse_score_larger.csv')
true_score_value.rename(columns={'Unnamed: 0': 'Image_ID'}, inplace=True)

In [132]:
new_data_final = data_final.iloc[:,:-6].merge(true_score_value[true_score_value['safety'].notna()].loc[:,['Image_ID', 'safety']], how='inner', on='Image_ID')

In [136]:
from scipy.stats import normaltest
import seaborn as sns
from scipy.stats import norm
import matplotlib.pyplot as plt

# Perform the normality test
stat, p_value = normaltest(new_data_final['safety'])
mu, std = norm.fit(new_data_final['safety'])


# Print the results
print(f'Statistic: {stat}, p-value: {p_value}')
print(f'mean: {mu}, std: {std}')

# Plot the distribution of 'Score'
# sns.histplot(data['Score'], kde=True)
# plt.title('Distribution of Scores')
# plt.xlabel('Score')
# plt.ylabel('Frequency')
# plt.show()

# Interpretation of the p-value
alpha = 0.05
if p_value > alpha:
    print('The null hypothesis cannot be rejected. The data follows a normal distribution.')
else:
    print('The null hypothesis is rejected. The data does not follow a normal distribution.')

Statistic: 1.6630262781862668, p-value: 0.43538998203520907
mean: 5.054397875836562, std: 0.8758323811397336
The null hypothesis cannot be rejected. The data follows a normal distribution.


In [140]:
# new_data_final.rename(columns={'safety': 'Score'}, inplace=True)
# new_data_final['Category'] = 'safety'
new_data_final.loc[:,'label'] = 0
new_data_final.loc[new_data_final[new_data_final['Score'] < mu - 0.5*std].index,'label'] = -1
new_data_final.loc[new_data_final[new_data_final['Score'] > mu + 0.5*std].index,'label'] = 1

In [163]:
new_data_final.head()

,GSV_idx,Image_ID,text_description,age,gender,location,event,text_description_short,Score,Category,label,GSV_path
0,503,50f42b7bfdc9f065f00013bd,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,"In the panoramic street view image, there are ...",3.922487,safety,-1,/data2/cehou/LLM_safety/PlacePulse2.0/photo_da...
1,508,50f42b7bfdc9f065f00013c4,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a street...,5.708965,safety,1,/data2/cehou/LLM_safety/PlacePulse2.0/photo_da...
2,514,50f42b7cfdc9f065f00013d0,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The street scene in the panoramic street view ...,5.377873,safety,0,/data2/cehou/LLM_safety/PlacePulse2.0/photo_da...
3,515,50f42b7cfdc9f065f00013d1,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a street...,3.863757,safety,-1,/data2/cehou/LLM_safety/PlacePulse2.0/photo_da...
4,522,50f42b7cfdc9f065f00013db,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,The panoramic street view image shows a street...,5.009854,safety,0,/data2/cehou/LLM_safety/PlacePulse2.0/photo_da...


In [162]:
new_data_final.to_csv('/data2/cehou/LLM_safety/img_text_data/baseline/tidyed/dataset_baseline_baseline_baseline_baseline_7850_withlabel.csv', index=False)

In [84]:
# data_config = dataset[0].split("_")
# save_filename = f"/data1/cehou_data/LLM_safety/img_text_data/dataset_{data_config[1]}_{data_config[2]}_{data_config[3]}_{data_config[4]}_{len(all_data)}.pkl"
# with open(save_filename, "wb") as f:
#     pickle.dump(all_data, f)
# print(f"save the data into {save_filename}")